#### kearkliy kutubxonalarni chaqirib olamiz

In [265]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

df = pd.read_csv("example.csv")

#### DataFrame haqida ma'lmotlarni kurib olamiz

In [266]:
df.shape

(40769, 32)

In [267]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40769 entries, 0 to 40768
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   40769 non-null  int64  
 1   Safety                       40769 non-null  int64  
 2   yacEPA                       15867 non-null  float64
 3   Drive                        40769 non-null  int64  
 4   FieldGoalDistance            902 non-null    float64
 5   airEPA                       15875 non-null  float64
 6   Field_Goal_Prob              40748 non-null  float64
 7   posteam                      38219 non-null  object 
 8   Penalty.Yards                40769 non-null  int64  
 9   FieldGoalResult              908 non-null    object 
 10  DefTeamScore                 38019 non-null  float64
 11  Tackler2                     4943 non-null   object 
 12  Away_WP_pre                  38223 non-null  float64
 13  Timeout_Indicato

#### ichida NaN qiymatlar bor ustunlarni kurib olamiz

In [268]:
count = len(df)
nan_columns = {}
for column in df.columns:
    nan_count = df[column].isna().sum()
    if nan_count > 0:
        nan_columns[column] = {
            'nan': nan_count,
            'not_nan': count - nan_count,
            'all': count,
            'percentage': round((nan_count / count) * 100, 2)}

for i in nan_columns:
    print(i, nan_columns[i])

yacEPA {'nan': 24902, 'not_nan': 15867, 'all': 40769, 'percentage': 61.08}
FieldGoalDistance {'nan': 39867, 'not_nan': 902, 'all': 40769, 'percentage': 97.79}
airEPA {'nan': 24894, 'not_nan': 15875, 'all': 40769, 'percentage': 61.06}
Field_Goal_Prob {'nan': 21, 'not_nan': 40748, 'all': 40769, 'percentage': 0.05}
posteam {'nan': 2550, 'not_nan': 38219, 'all': 40769, 'percentage': 6.25}
FieldGoalResult {'nan': 39861, 'not_nan': 908, 'all': 40769, 'percentage': 97.77}
DefTeamScore {'nan': 2750, 'not_nan': 38019, 'all': 40769, 'percentage': 6.75}
Tackler2 {'nan': 35826, 'not_nan': 4943, 'all': 40769, 'percentage': 87.88}
Away_WP_pre {'nan': 2546, 'not_nan': 38223, 'all': 40769, 'percentage': 6.24}
TimeSecs {'nan': 27, 'not_nan': 40742, 'all': 40769, 'percentage': 0.07}
Passer {'nan': 24073, 'not_nan': 16696, 'all': 40769, 'percentage': 59.05}
TwoPointConv {'nan': 40716, 'not_nan': 53, 'all': 40769, 'percentage': 99.87}
PassOutcome {'nan': 24018, 'not_nan': 16751, 'all': 40769, 'percentage'

#### dataframemizda haddan tashqari kup NaN qiymatlarga ega ustunlarimiz mavjud hozirni buni qayta tuldirishning iliji yuq, shuning uchun ushbu ustunlarni tashlab yuborishga tug'ri keladi, pasdagi kod orqaliy 10% dan kup NaN qiymatlarga ega ustunlarni tashlab yuboramiz!

In [269]:
percent = df.isna().mean() * 100
clean_df = df.loc[:, percent < 10.0]

##### qolgan NaN qiymati bor ustunlarni listga saqlab olamiz

In [270]:
non_columns = df.columns[df.isna().any()].tolist()

In [271]:
non_columns

['yacEPA',
 'FieldGoalDistance',
 'airEPA',
 'Field_Goal_Prob',
 'posteam',
 'FieldGoalResult',
 'DefTeamScore',
 'Tackler2',
 'Away_WP_pre',
 'TimeSecs',
 'Passer',
 'TwoPointConv',
 'PassOutcome',
 'PlayTimeDiff',
 'RunGap',
 'Opp_Safety_Prob',
 'No_Score_Prob']

##### Yangi DataFramemizning holatini kuramiz

In [272]:
clean_df.shape

(40769, 23)

#### object bulgan ustunlarimiznin ajratib olamiz


In [273]:
object_col = clean_df.select_dtypes(include=['object']).columns


#### Object va float ustunlarni ikta DataFramega ajratib olamiz

In [274]:
train_object_df = clean_df[object_col].reset_index(drop=True)
train_num_df = clean_df.drop(columns=object_col)
# object ustunlarimizga mod qiymatni quyamzi
for i in object_col:
    train_object_df.fillna(clean_df[i].mode()[0], inplace=True)

#### Menda ushbu ustunlar buyicha ma'lumot yuqligi uchun bu ustunlarni IterativeImputer orqaliy taxminiy tuldiramiz, agarda ustunlar haqida ma'lumot bulganda biz mean / mod lardan foydalanishimiz mumkin edi!



In [275]:
iterative_imputer = IterativeImputer(random_state=0)
new_df = pd.DataFrame(iterative_imputer.fit_transform(train_num_df), columns=train_num_df.columns)

#### Object va Float ustunlarni birlashtiramiz

In [276]:
for i in object_col:
    new_df[i] = train_object_df[i]

#### Yangi Dataframemizni holatini kurib olamiz

In [277]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40769 entries, 0 to 40768
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   40769 non-null  float64
 1   Safety                       40769 non-null  float64
 2   Drive                        40769 non-null  float64
 3   Field_Goal_Prob              40769 non-null  float64
 4   Penalty.Yards                40769 non-null  float64
 5   DefTeamScore                 40769 non-null  float64
 6   Away_WP_pre                  40769 non-null  float64
 7   Timeout_Indicator            40769 non-null  float64
 8   ydsnet                       40769 non-null  float64
 9   TimeSecs                     40769 non-null  float64
 10  Touchdown                    40769 non-null  float64
 11  PlayTimeDiff                 40769 non-null  float64
 12  RushAttempt                  40769 non-null  float64
 13  Accepted.Penalty

In [278]:
new_df.shape

(40769, 23)

#### NaN qiymatlar uchun belgi quyib ketamiz buning uchun bizga non_columns listimizdan foydalanamiz

In [279]:
for column in non_columns:
    nan_count = df[column].isna().sum()
    if nan_count > 0:
        new_df[column + '_IsNaN'] = df[column].isna()

##### uzgartirishlardan sungi ikta Dataframelarni farqini kuramiz

In [280]:
df.describe() - new_df.describe()

,Accepted.Penalty,AwayTimeouts_Remaining_Post,Away_WP_pre,Challenge.Replay,DefTeamScore,Drive,FieldGoalDistance,Field_Goal_Prob,No_Score_Prob,Opp_Safety_Prob,...,Season,TimeSecs,TimeUnder,Timeout_Indicator,Touchdown,Unnamed: 0,airEPA,qtr,yacEPA,ydsnet
count,0.0,0.0,-2546.000000,0.0,-2750.000000,0.0,NaN,-21.000000,-2.100000e+01,-2.100000e+01,...,0.0,-27.000000,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
mean,0.0,0.0,0.000382,0.0,-0.342699,0.0,NaN,-0.000034,4.107565e-07,7.316945e-07,...,0.0,0.281351,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
std,0.0,0.0,0.009041,0.0,0.114335,0.0,NaN,0.000025,2.596291e-05,5.005464e-07,...,0.0,-0.001580,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
min,0.0,0.0,0.000000,0.0,0.000000,0.0,NaN,0.000000,1.088262e-01,2.023729e-03,...,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
25%,0.0,0.0,-0.020290,0.0,0.000000,0.0,NaN,-0.000041,-1.153595e-06,0.000000e+00,...,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
50%,0.0,0.0,0.004679,0.0,0.000000,0.0,NaN,-0.000050,-2.943340e-05,3.025881e-07,...,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
75%,0.0,0.0,0.022525,0.0,0.000000,0.0,NaN,-0.000064,-1.470572e-04,9.350682e-07,...,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0
max,0.0,0.0,0.000000,0.0,0.000000,0.0,NaN,0.000000,0.000000e+00,0.000000e+00,...,0.0,0.000000,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0


#### Yangi DataFrame

In [281]:
new_df.head()

,Unnamed: 0,Safety,Drive,Field_Goal_Prob,Penalty.Yards,DefTeamScore,Away_WP_pre,Timeout_Indicator,ydsnet,TimeSecs,...,Tackler2_IsNaN,Away_WP_pre_IsNaN,TimeSecs_IsNaN,Passer_IsNaN,TwoPointConv_IsNaN,PassOutcome_IsNaN,PlayTimeDiff_IsNaN,RunGap_IsNaN,Opp_Safety_Prob_IsNaN,No_Score_Prob_IsNaN
0,244485.0,0.0,18.0,0.309404,0.0,10.0,0.774353,0.0,11.0,939.0,...,True,False,False,False,True,False,False,True,False,False
1,115340.0,0.0,22.0,0.198142,0.0,14.0,0.943964,0.0,24.0,407.0,...,True,False,False,True,True,True,False,True,False,False
2,68357.0,0.0,8.0,0.000000,0.0,13.0,0.634693,0.0,80.0,1823.0,...,True,False,False,True,True,True,False,True,False,False
3,368377.0,0.0,24.0,0.258227,0.0,31.0,0.064005,0.0,25.0,528.0,...,True,False,False,False,True,False,False,True,False,False
4,384684.0,0.0,11.0,0.199771,0.0,24.0,0.071526,0.0,0.0,2355.0,...,True,False,False,True,True,True,False,False,False,False


##### Datani tozaladik Haddan tashqari kup nan qiymatlari bor ustunlar tashlab yuborildi, qatorlarga zarar yitkazilmadi lekin datframening ba'zi kursatgichlari oz darajada uzgardi! Kod qisq bulishiga qaramdan anch kup vaqt va kup izlanish sarflandi